In [43]:
pwd

'D:\\project\\machine_failuire'

In [ ]:
import os 
os.chdir("..")

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df=pd.read_csv("data\data.csv")
df

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
0,0,7,7,1,6,6,36,3,1,1
1,190,1,3,3,5,1,20,4,1,0
2,31,7,2,2,6,1,24,6,1,0
3,83,4,3,4,5,1,28,6,1,0
4,640,7,5,6,4,0,68,6,1,0
...,...,...,...,...,...,...,...,...,...,...
939,0,7,7,1,6,4,73,6,24,1
940,0,7,5,2,6,6,50,6,24,1
941,0,3,6,2,7,5,43,6,24,1
942,0,6,6,2,5,6,46,7,24,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   footfall     944 non-null    int64
 1   tempMode     944 non-null    int64
 2   AQ           944 non-null    int64
 3   USS          944 non-null    int64
 4   CS           944 non-null    int64
 5   VOC          944 non-null    int64
 6   RP           944 non-null    int64
 7   IP           944 non-null    int64
 8   Temperature  944 non-null    int64
 9   fail         944 non-null    int64
dtypes: int64(10)
memory usage: 73.9 KB


In [ ]:
df.describe()

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
count,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000
mean,306.381356,3.727754,4.325212,2.939619,5.394068,2.842161,47.043432,4.565678,16.331568,0.416314
std,1082.606745,2.677235,1.438436,1.383725,1.269349,2.273337,16.423130,1.599287,5.974781,0.493208
min,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,19.000000,1.000000,1.000000,0.000000
25%,1.000000,1.000000,3.000000,2.000000,5.000000,1.000000,34.000000,3.000000,14.000000,0.000000
50%,22.000000,3.000000,4.000000,3.000000,6.000000,2.000000,44.000000,4.000000,17.000000,0.000000
75%,110.000000,7.000000,6.000000,4.000000,6.000000,5.000000,58.000000,6.000000,21.000000,1.000000
max,7300.000000,7.000000,7.000000,7.000000,7.000000,6.000000,91.000000,7.000000,24.000000,1.000000


In [5]:
X=df.drop(["fail"],axis=1)
y=df.fail

In [38]:
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn 
from torch.utils.data import DataLoader ,TensorDataset
import mlflow
import mlflow.pytorch
import optuna

In [86]:
sclaer=StandardScaler()
X_scl=sclaer.fit_transform(X)
X_train,X_test,y_train,y_test = train_test_split(X_scl,y,test_size=0.2,random_state=42,shuffle=True)

In [87]:
X_train =pd.DataFrame(X_train)
X_test =pd.DataFrame(X_test)
X_train

,0,1,2,3,4,5,6,7,8
0,-0.262821,-0.271975,1.860497,-1.40248,0.477610,0.949698,-1.160164,-0.353894,-0.390442
1,-0.283153,0.849179,-0.226207,0.04366,0.477610,-0.370648,2.677922,-2.230725,-1.897572
2,-0.283153,0.849179,0.469361,0.04366,0.477610,0.949698,1.033028,-1.605114,-0.222983
3,-0.274836,1.222896,0.469361,-1.40248,-1.098836,0.949698,1.946858,1.522937,-0.055524
4,-0.283153,1.222896,0.469361,-0.67941,-1.098836,0.509582,0.301964,-0.979504,0.111935
...,...,...,...,...,...,...,...,...,...
750,0.077279,1.222896,-0.226207,1.48980,-1.887059,-0.810763,-0.246334,-0.979504,-1.562654
751,-0.256352,-1.019410,0.469361,-0.67941,-1.098836,-0.370648,-1.342930,0.271716,-0.390442
752,-0.261897,0.475461,1.164929,0.04366,0.477610,1.389813,0.119198,0.897327,1.116688
753,-0.181493,1.222896,-0.226207,-0.67941,0.477610,0.509582,1.764092,-0.353894,0.111935


In [88]:
X_train = torch.tensor(X_train.values,dtype=torch.float32)
X_test = torch.tensor(X_test.values,dtype=torch.float32)
y_train = torch.tensor(y_train.values,dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values,dtype=torch.float32).unsqueeze(1)

In [6]:
X.columns

Index(['footfall', 'tempMode', 'AQ', 'USS', 'CS', 'VOC', 'RP', 'IP',
       'Temperature'],
      dtype='object')

In [90]:
train_ds=TensorDataset(X_train,y_train)
test_ds=TensorDataset(X_test,y_test)

In [91]:
trainloader=DataLoader(train_ds)

In [56]:
class MYNN(nn.Module):
    def __init__(self, hidden, dropout):
        super().__init__()
        self.net= nn.Sequential(
            nn.Linear(X_train.shape[1],hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.net(x)

In [73]:
from tqdm import tqdm

In [74]:
def train_model(params):

    model = MYNN(params["hidden"], params["dropout"])
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

    ########## train
    for epoch in range(params["epochs"]):
        model.train()
        for xb, yb in tqdm(trainloader,total=len(trainloader)):
            pred = model(xb)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    ########## test
    model.eval()
    with torch.no_grad():
        test_pred = model(X_test)
        test_loss = loss_fn(test_pred,y_test)
        test_acc = ((test_pred > 0.5).float()==y_test).float().mean().item()

    return model,test_acc,test_loss

In [ ]:
import mlflow.pytorch


def objective(trail):

    params={
        "hidden" : trail.suggest_int("hidden",16,64),
        "dropout" : round(trail.suggest_float("dropout",0.2,0.5),1),
        "lr" : round(trail.suggest_float("lr",0.01,0.1),2),
        "epochs" : trail.suggest_int("epochs",5,15)
    }

    with mlflow.start_run():

        model,test_acc,test_loss = train_model(params)
        
        mlflow.log_params(params)
        mlflow.log_metric("accuracy",test_acc)
        mlflow.pytorch.log_model(model)
    return test_loss



In [ ]:
mlflow.set_experiment("machine_failuire")
study =optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=20)

2025/07/09 00:25:27 INFO mlflow.tracking.fluent: Experiment with name 'machine_failuire' does not exist. Creating a new experiment.
[I 2025-07-09 00:25:27,194] A new study created in memory with name: no-name-565674ea-64d5-4b79-ad5c-910985383d6d
2025/07/09 00:25:28 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging l